In [4]:
# 엑셀파일이 'phenotype mappingv3.xlsx', '1_검증.csv' 이렇게 2개가 있어
#  'phenotype mappingv3.xlsx' 파일은 UMLSID, Group, GroupID 이렇게 3개의 컬럼이 존재하고
# '1_검증.csv' 파일은 Phenotype, Network_score, z-score, P-value, UMLSID 이렇게 5개의 컬럼이 존재해
# 여기서 '1_검증.csv' 파일 내에 있는 UMLSID의 데이터와  'phenotype mappingv3.xlsx' 파일 내에 있는 UMLSID의 데이터가 일치하면 Group, GroupID 값을 가져와서 똑같이 써주고, 아니면 None 값을 써주고 싶어
# 그래서 최종적으로 '1_검증.csv' 파일내에 Phenotype, Network_score, z-score, P-value, UMLSID, Group, GroupID 이렇게 7개의 컬럼이 존재하게 만들고 싶어
# 이렇게 코드 짜줘

import numpy as np
import pandas as pd
import configparser
import psycopg2
import os
import ast
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
from collections import defaultdict
from statistics import harmonic_mean
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# phenotype_mapping_file='/data/home/sss2061/식약처보고서/서울과기대/phenotype mappingv3.xlsx'

# '1_검증.csv' 파일을 읽어옵니다.

number=2 #수정 필요
validation_df = pd.read_csv('result/{}차/{}_검증.csv'.format(number,number)) #수정 필요

# 'phenotype mappingv3.xlsx' 파일을 읽어옵니다.
mapping_df = pd.read_excel('phenotype mappingv3.xlsx', engine='openpyxl')

# '1_검증.csv' 파일 내의 UMLSID와 'phenotype mappingv3.xlsx' 파일 내의 UMLSID를 기반으로 병합합니다.
merged_df = validation_df.merge(mapping_df, on='UMLSID', how='left')

# Group 및 GroupID가 없는 경우 None으로 채웁니다.
merged_df['Group'].fillna('None', inplace=True)
merged_df['GroupID'].fillna('None', inplace=True)

# 'Network_score' 컬럼의 데이터를 추출합니다.
network_scores = merged_df['Network_score']

# Min-Max Scaling을 0부터 1로 설정하여 적용합니다.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_scores = scaler.fit_transform(network_scores.values.reshape(-1, 1))  # 2D 배열로 변환 후 스케일링

# 스케일링된 값을 'scaling_score' 컬럼에 추가합니다.
merged_df['scaling_score'] = scaled_scores

# 'P-value'의 값을 조정합니다.
def adjust_pvalue(value):
    if value == 999 or value == 1000:
        return 1
    else:
        return (value+1) / 1000

merged_df['P-value'] = merged_df['P-value'].apply(adjust_pvalue)

cols_to_drop = [col for col in merged_df.columns if col.startswith("Unnamed")]
merged_df = merged_df.drop(columns=cols_to_drop)

hpo_df=pd.read_csv("Data/[original] HPO phenotype data.csv")

merged_df=pd.merge(merged_df, hpo_df, left_on='Phenotype', right_on='phenotype name', how='left')

final_df=merged_df[['Phenotype','Network_score','z-score','P-value','UMLSID','Group','GroupID','scaling_score', 'hpo_id']]

# 수정된 DataFrame을 새로운 엑셀 파일로 저장합니다.
final_df.to_csv('result/{}차/{}_검증_with_scaling.csv'.format(number,number), index=False)  # 저장할 파일명을 지정해주세요.

In [ ]:
# 백업본

# 엑셀파일이 'phenotype mappingv3.xlsx', '1_검증.csv' 이렇게 2개가 있어
#  'phenotype mappingv3.xlsx' 파일은 UMLSID, Group, GroupID 이렇게 3개의 컬럼이 존재하고
# '1_검증.csv' 파일은 Phenotype, Network_score, z-score, P-value, UMLSID 이렇게 5개의 컬럼이 존재해
# 여기서 '1_검증.csv' 파일 내에 있는 UMLSID의 데이터와  'phenotype mappingv3.xlsx' 파일 내에 있는 UMLSID의 데이터가 일치하면 Group, GroupID 값을 가져와서 똑같이 써주고, 아니면 None 값을 써주고 싶어
# 그래서 최종적으로 '1_검증.csv' 파일내에 Phenotype, Network_score, z-score, P-value, UMLSID, Group, GroupID 이렇게 7개의 컬럼이 존재하게 만들고 싶어
# 이렇게 코드 짜줘

import numpy as np
import pandas as pd
import configparser
import psycopg2
import os
import ast
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
from collections import defaultdict
from statistics import harmonic_mean

# phenotype_mapping_file='/data/home/sss2061/식약처보고서/서울과기대/phenotype mappingv3.xlsx'

# '1_검증.csv' 파일을 읽어옵니다.

number=2
validation_df = pd.read_csv('result/{}차/{}_검증.csv'.format(number,number))

# 'phenotype mappingv3.xlsx' 파일을 읽어옵니다.
mapping_df = pd.read_excel('phenotype mappingv3.xlsx', engine='openpyxl')

# '1_검증.csv' 파일 내의 UMLSID와 'phenotype mappingv3.xlsx' 파일 내의 UMLSID를 기반으로 병합합니다.
merged_df = validation_df.merge(mapping_df, on='UMLSID', how='left')

# Group 및 GroupID가 없는 경우 None으로 채웁니다.
merged_df['Group'].fillna('None', inplace=True)
merged_df['GroupID'].fillna('None', inplace=True)

# 결과를 'merged_1_검증.csv' 파일로 저장합니다.
merged_df.to_csv('merged_14_검증.csv', index=False)

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 'merged_1_검증.xlsx' 파일을 읽어옵니다.
df = pd.read_csv('merged_14_검증.csv')  # 파일 경로를 확인해주세요.

# 'Network_score' 컬럼의 데이터를 추출합니다.
network_scores = df['Network_score']

# Min-Max Scaling을 0부터 1로 설정하여 적용합니다.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_scores = scaler.fit_transform(network_scores.values.reshape(-1, 1))  # 2D 배열로 변환 후 스케일링

# 스케일링된 값을 'scaling_score' 컬럼에 추가합니다.
df['scaling_score'] = scaled_scores

# 'P-value'의 값을 조정합니다.
def adjust_pvalue(value):
    if value == 999 or value == 1000:
        return 1
    else:
        return (value+1) / 1000

df['P-value'] = df['P-value'].apply(adjust_pvalue)

# "Unnamed"로 시작하는 컬럼을 찾아 삭제합니다.
cols_to_drop = [col for col in df.columns if col.startswith("Unnamed")]
df = df.drop(columns=cols_to_drop)

# 수정된 DataFrame을 새로운 엑셀 파일로 저장합니다.
df.to_csv('merged_14_검증_with_scaling.csv', index=False)  # 저장할 파일명을 지정해주세요.

In [1]:
# number=1
# text=["adenine", "canavanin", "malate", "isoleucine", "phenylalanine", "tyrosine", "isocitrate", "glutamine", "tryptophan","L-malate", 
#       "theogallin", "trans-aconitate", "cis-aconitate", "threonate", "benzoic acid", "choline", "citric acid", "coumarin", "phloroglucinol", "gallic acid", 
#       "citrullin", "pipecolic acid", "vitamin B", "pantothenic acid", "DL-valine", "vanillin", "theophylline", "caffeine", "protocatechuic aldehyde", "adenosine", 
#       "EGCG", "gallocatechin", "N-acetylglutamate", "loliolide", "procyanidin B2", "3H-proline", "dihydromyricetin", "beta-alanine betaine", "epicatechin gallate (ECG", "5'-methylthioadenosine", 
#       "epiafzelechin", "ferulic acid", "p-coumaric acid", "caffeic acid", "chlorogenic acid", "quercetin", "alpha-isopropylmalate", "neochlorogenic acid", "rutin", "hyperoside", 
#       "4-p-coumaroylquinic acid", "astragalin", "quercetin 7-O-glucoside", "p-coumaroylquinic acid", "procyanidin B1"]

# number=2
# text=["gamma-aminobutyric acid (GABA", "canavanin", "sucrose", "folate", "guanosine", "vanillic acid", "ononin", 
#       "formononetin", "calycosin", "sissotrin", "afrormosin", "pratensein"]

# number=3
# text=["protocatechuic acid", "gamma-aminobutyric acid (GABA", "choline", "citric acid", "malate", "asparagine", 
#       "quinic acid", "maltol", "nodakenetin", "nodakenin", "columbianetin", "decursin", "ferulic acid", "caffeic acid",
#       "aegelinol", "cis-chlorogenic acid", "chlorogenic acid", "esculetin"]

# number=4
# text=["piperine", "piperanine", "piperlonguminine", "retrofractamide B", "dehydropipernonaline", "Methyl piperate", "pipernonaline", "Retrofractamide A", "retrofractamide C"]

# number=5
# text=["apocynin", "resacetophenone", "4-hydroxyacetophenone", "2,5-dihydroxyacetophenone", "physcion"]

# number=6
# text=["protocatechuic acid", "p-hydroxybenzaldehyde", "choline", "gluconate", "myo-inositol", "thymine", "uracil", "vanillin", "vanillic acid", "protocatechuic aldehyde", "trans-aconitate", "ferulic acid", "caffeic acid", "chlorogenic acid", "linoleic acid", "neochlorogenic acid", "xanthatin", "cynarin", "cynarine"]

# number=7
# text=["atractylenolide III", "vanillic acid", "esculetin", "elemicin", "adenosine"]


1


In [82]:
#결과 후처리

from tqdm import tqdm_notebook
import numpy as np
import pickle
import pandas as pd
import configparser
import psycopg2
import os
import ast
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
import copy
from collections import defaultdict
from statistics import harmonic_mean
import pickle

number=7

with open("result/{}차/output_dict.pickle".format(number),"rb") as fi:
    output_dict = pickle.load(fi)
    
# validation_df = pd.read_csv('result/{}차/{}_검증.csv'.format(number,number))
inferred_phenotype2 = pd.read_csv('result/{}차/{}_network_score.csv'.format(number,number))

phenotype_rwr_score_dict = pd.Series(inferred_phenotype2.Network_score.values, index=inferred_phenotype2.Phenotype).to_dict()

temp=list(phenotype_rwr_score_dict.keys())
for i in temp:
    if np.isnan(phenotype_rwr_score_dict[i])==True:
        phenotype_rwr_score_dict.pop(i)

Phenotype_score_Phenotype=list(phenotype_rwr_score_dict.keys())
Phenotype_score_Score=list(phenotype_rwr_score_dict.values())

temp2=list(output_dict.keys())
for i in temp2:
    if np.isnan(output_dict[i][0])==True:
        output_dict.pop(i)
        
P_value_Phenotype=list(output_dict.keys())
P_value_Score=list(output_dict.values())


In [83]:
count=0
z_score_list=[]
outlier_list=[] #제거가능
P_value_list=[]
rank_number=0

for i in range(len(P_value_Score)):
    Numberlist=P_value_Score[i].copy()
    tmp1=P_value_Phenotype[i]
    tmp2=Phenotype_score_Phenotype.index(tmp1)
    tmp3=Phenotype_score_Score[tmp2]
    Numberlist.append(tmp3)
    
    
    # 평균 계산
    mean = calculate_mean(Numberlist)

    # 표준편차 계산
    std_deviation = calculate_standard_deviation(Numberlist)

    z_score=(Phenotype_score_Score[i]-mean)/std_deviation
    z_score_list.append(z_score)
    if z_score >=-2.58 and z_score<=2.58: #제거가능
        outlier=1
        count=count+1
    else:
        outlier=0 
    outlier_list.append(outlier) #제거가능

    Numberlist.sort(reverse=True)

    rank=Numberlist.index(tmp3) #순위찾기
    if rank<5:
        rank_number=rank_number+1
    P_value_list.append(rank)

# postgresql DB 연동
config = configparser.ConfigParser()
config.read('db_config.ini')
conn=psycopg2.connect(host="168.131.30.66", dbname="coconut",user="dbuser",password="jnudl1") #데이터이름변경
conn_cur = conn.cursor()
#------------------------------------
phenotype_umlsid = []
# phenotype_list를 순회하면서 각 이름에 대한 UMLS ID를 가져옴
for phen_name in Phenotype_score_Phenotype:
    # 이름에 단일 따옴표가 포함된 경우 이스케이프
#     escaped_name = phen_name.replace("'", "''")
    
#     conn_cur.execute(f"SELECT umlsid FROM phenotype WHERE name = '{escaped_name}'")
    conn_cur.execute("SELECT umlsid FROM phenotype WHERE name = %s",(phen_name,))
    umlsid_result = conn_cur.fetchone()
    
    # UMLS ID가 존재하면 리스트에 추가
    if umlsid_result:
        phenotype_umlsid.append(umlsid_result[0])
    else:
        # UMLS ID가 없는 경우에 대한 처리 (예: None 또는 다른 기본값)
        phenotype_umlsid.append(None)

# 결과를 DataFrame으로 만들어 저장
phenotype_umlsid_df = pd.DataFrame({'Name': Phenotype_score_Phenotype, 'UMLSID': phenotype_umlsid})
print(phenotype_umlsid_df)

#------------------------

Phenotype_score = pd.DataFrame({
    'Phenotype': Phenotype_score_Phenotype,
    'Network_score': Phenotype_score_Score,
    'z-score': z_score_list,
    'P-value': P_value_list,
    'UMLSID': phenotype_umlsid
})

Phenotype_score=Phenotype_score.sort_values(by='P-value',ascending=True)

#     # Compound ID에 해당하는 inferred phenotype.csv 생성
# if not os.path.exists('compound 결과'):
#     os.makedirs('compound 결과')
Phenotype_score.to_csv('result/{}차/{}_검증.csv'.format(number,number),index=False, encoding='UTF-8')
    
end = time.time()
sec = (end - start)
result = datetime.timedelta(seconds=sec)
print("걸린 시간 : ",result)
print(f"걸린 시간 : {end - start:.5f} sec")

# 엑셀파일이 'phenotype mappingv3.xlsx', '1_검증.csv' 이렇게 2개가 있어
#  'phenotype mappingv3.xlsx' 파일은 UMLSID, Group, GroupID 이렇게 3개의 컬럼이 존재하고
# '1_검증.csv' 파일은 Phenotype, Network_score, z-score, P-value, UMLSID 이렇게 5개의 컬럼이 존재해
# 여기서 '1_검증.csv' 파일 내에 있는 UMLSID의 데이터와  'phenotype mappingv3.xlsx' 파일 내에 있는 UMLSID의 데이터가 일치하면 Group, GroupID 값을 가져와서 똑같이 써주고, 아니면 None 값을 써주고 싶어
# 그래서 최종적으로 '1_검증.csv' 파일내에 Phenotype, Network_score, z-score, P-value, UMLSID, Group, GroupID 이렇게 7개의 컬럼이 존재하게 만들고 싶어
# 이렇게 코드 짜줘


# 'phenotype mappingv3.xlsx' 파일을 읽어옵니다.
mapping_df = pd.read_excel('phenotype mappingv3.xlsx', engine='openpyxl')

# '1_검증.csv' 파일 내의 UMLSID와 'phenotype mappingv3.xlsx' 파일 내의 UMLSID를 기반으로 병합합니다.
merged_df = Phenotype_score.merge(mapping_df, on='UMLSID', how='left')

# Group 및 GroupID가 없는 경우 None으로 채웁니다.
merged_df['Group'].fillna('None', inplace=True)
merged_df['GroupID'].fillna('None', inplace=True)

# 'Network_score' 컬럼의 데이터를 추출합니다.
network_scores = merged_df['Network_score']

# Min-Max Scaling을 0부터 1로 설정하여 적용합니다.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_scores = scaler.fit_transform(network_scores.values.reshape(-1, 1))  # 2D 배열로 변환 후 스케일링

# 스케일링된 값을 'scaling_score' 컬럼에 추가합니다.
merged_df['scaling_score'] = scaled_scores

# 'P-value'의 값을 조정합니다.
def adjust_pvalue(value):
    if value == 999 or value == 1000:
        return 1
    else:
        return (value+1) / 1000

merged_df['P-value'] = merged_df['P-value'].apply(adjust_pvalue)

cols_to_drop = [col for col in merged_df.columns if col.startswith("Unnamed")]
merged_df = merged_df.drop(columns=cols_to_drop)

hpo_df=pd.read_csv("Data/[original] HPO phenotype data.csv")

merged_df=pd.merge(merged_df, hpo_df, left_on='Phenotype', right_on='phenotype name', how='left')

final_df=merged_df[['Phenotype','Network_score','z-score','P-value','UMLSID','Group','GroupID','scaling_score', 'hpo_id']]

# 수정된 DataFrame을 새로운 엑셀 파일로 저장합니다.
final_df.to_csv('result/{}차/{}_검증_with_scaling.csv'.format(number,number), index=False)  # 저장할 파일명을 지정해주세요.

                                         Name    UMLSID
0                              Arachnodactyly  C0003706
1                                      Stroke      None
2                                Hypohidrosis  C0020620
3                                Hyperkalemia  C0020461
4     Decreased circulating aldosterone level  C0857899
...                                       ...       ...
6704    Abnormal full-field electroretinogram  C4072956
6705  Abnormal fundus fluorescein angiography  C4073074
6706    Abnormal multifocal electroretinogram  C4072958
6707                      Mongolian blue spot      None
6708                           Absent malleus      None

[6709 rows x 2 columns]
걸린 시간 :  0:29:21.689884
걸린 시간 : 1761.68988 sec
